In [ ]:
GOOGLE = True
if GOOGLE:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title import
import sys
sys.path.append('/content/drive/MyDrive/torch_code/')
from shared.data_loader import *
from shared.utils import *

In [ ]:
# @title GPU code
import torch
import torch.nn.functional as F

# Define the device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

Good to go!


In [ ]:
# @title EEG DATA
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
import pandas as pd

class EEG_Data(Dataset):

    def __init__(self, root_dir, split, preprocess=lambda x,y:train_data_prep(x,y,2,2,True), transform=None, label_dict=None):
        """
        Initialize the eeg dataset with the root directory for the images,
        the split (train/val/test), an optional data transformation,
        and an optional label dictionary.

        Args:
            root_dir (str): Root directory for the eeg images.
            split (str): Split to use ('train', 'val', or 'test').
            transform (callable, optional): Optional data transformation to apply to the images.
            label_dict (dict, optional): Optional dictionary mapping integer labels to class names.
        """
        assert split in ['train', 'val', 'test']
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.datastorch = []
        self.labels = []
        self.label_dict = ["Cue Onset left", "Cue Onset right", "Cue onset foot", "Cue onset tongue"]

        ################# Your Implementations #################################
        if self.split == 'train':
            # First generating the training and validation indices using random splitting
            X_train_valid = np.load(self.root_dir+"X_train_valid.npy")
            y_train_valid = np.load(self.root_dir+"y_train_valid.npy")

            np.random.seed(0)
            data_length = len(X_train_valid)

            ind_valid = np.random.choice(data_length, int(data_length*0.1), replace=False)
            ind_train = np.array(list(set(range(data_length)).difference(set(ind_valid))))

            # Creating the training and validation sets using the generated indices
            (x_train, x_valid) = X_train_valid[ind_train], X_train_valid[ind_valid]
            (y_train, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]

            if preprocess is not None:
                x_train,y_train = preprocess(x_train,y_train)

            self.datas = torch.from_numpy(x_train)
            self.labels = [int(i-769) for i in torch.from_numpy(y_train)]

        if self.split == 'val':
            # First generating the training and validation indices using random splitting
            X_train_valid = np.load(self.root_dir+"X_train_valid.npy")
            y_train_valid = np.load(self.root_dir+"y_train_valid.npy")

            data_length = len(X_train_valid)

            np.random.seed(0)
            ind_valid = np.random.choice(data_length, int(data_length*0.1), replace=False)
            ind_train = np.array(list(set(range(data_length)).difference(set(ind_valid))))

            # Creating the training and validation sets using the generated indices
            (x_train, x_valid) = X_train_valid[ind_train], X_train_valid[ind_valid]
            (y_train, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]

            if preprocess is not None:
                x_valid,y_valid = preprocess(x_valid,y_valid)

            self.datas = torch.from_numpy(x_valid)
            self.labels = [int(i-769) for i in torch.from_numpy(y_valid)]

        if self.split == 'test':
            x_test = np.load(self.root_dir+"X_test.npy")
            # x_test = test_data_prep(x_test_og)  # (2115, 1)  vals from 0-8 for participant
            if preprocess is not None:
                x_test = preprocess(x_test)
            y_test = np.load(self.root_dir+"y_test.npy")  # (443, 1)
            self.datas = torch.from_numpy(x_test)
            self.labels = [int(i-769) for i in torch.from_numpy(y_test)]

        ################# End of your Implementations ##########################

    def __len__(self):
        """
        Return the number of images in the dataset.

        Returns:
            int: Number of images in the dataset.
        """
        dataset_len = 0
        ################# Your Implementations #################################
        # Return the number of images in the dataset
        dataset_len = len(self.datas)
        ################# End of your Implementations ##########################
        return dataset_len

    def __getitem__(self, idx):
        """
        R10140    idx (int): Index of the image to retrieve.

        Returns:
            tuple: Tuple containing the image and its label.
        """
        ################# Your Implementations #################################
        # Load and preprocess image using self.root_dir,
        # self.filenames[idx], and self.transform (if specified)

        data = self.datas[idx]
        label = self.labels[idx]

        if self.transform:
            data = self.transform(data)
        ################# End of your Implementations ##########################
        return data, label

In [ ]:
# @title Convolution

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

class ConvBlock(nn.Module):
    def __init__(self, input_size, output_size, kernel_size, dropout=0.6):
        super().__init__()
        padding = (kernel_size-1)//2
        self.conv_block = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=output_size, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, padding=1),
            nn.BatchNorm1d(output_size),
            nn.Dropout(dropout)
        )
    def forward(self, x, **kwargs):
        # keey the residual connection here
        return self.conv_block.forward(x)

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 3, downsample=False, dropout=0.6):
        '''
        Initialize a basic residual block.
        Depending on whether downsample is True, there are two types of basic blocks in Resnet 18,
        one to downsample the input and the other maintains the same size.
        You can use Pytorch's functions.

        Input and output shapes of each layer:
        1) conv1 (3*3 kernel, no bias): (batch_size, in_channels, H, W) -> (batch_size, out_channels, H, W) if downsample=False
           conv1 (3*3 kernel, no bias): (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) if downsample=True, with stride of 2
        2) conv2 (3*3 kernel, no bias): (batch_size, out_channels, H, W) -> (batch_size, out_channels, H, W)
        3) conv3 (optional) if downsample=True (1*1 kernel, no bias): (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) with stride of 2
        '''
        super().__init__()
        ################# Your Implementations #################################
        self.downsample = downsample
        if (in_channels != out_channels):
           self.downsample = True
        self.kernel_size = kernel_size
        self.dropout = dropout
        # self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, bias=False, padding=1) #(128 - 3 + 2*1)//2 + 1
        self.conv2 = nn.Sequential(
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias=False),
                        nn.BatchNorm1d(out_channels))
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.bn3 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.relu2 = nn.ReLU()
        if self.downsample:
          self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False) #(128 - 3 + 2*1)//2 + 1
          self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1, bias=False) #(128 - 3 + 2*1)//2 + 1
        else:
          self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False) #(128 - 3 + 2*1)//2 + 1

        ################# End of your Implementations ##########################

    def forward(self, x):
        """
        Forward pass of the basic residual block.
        The input tensor 'x' should pass through the following layers:
        1) conv1: (batch_size, in_channels, H, W) -> (batch_size, out_channels, H, W)
           conv1: (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) if downsample=True
        2) Apply batch normalization after conv1.
        3) Apply relu activation.
        4) conv2: (batch_size, out_channels, H, W) -> (batch_size, out_channels, H, W)
        5) Apply batch normalization after conv2.
        6) (Optional) if downsample=True, conv3: (batch_size, in_channels, H, W) -> (batch_size, out_channels, H//2, W//2) on the original input
        7) (Optional) if downsample=True, apply batch normalization after conv3.
        8) Add the residual value to the original input
        9) Apply relu activation in the end.
        """
        ################# Your Implementations #################################
        # TODO: Implement the forward pass of the basic residual block.
        og=x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample:
          og = self.conv3(og)
          og = self.bn3(og)
        x += og
        x =self.relu2(x)
        x = nn.Dropout(self.dropout)(x)
        ################# End of your Implementations ##########################
        return x

fc_block = lambda i, o : nn.Linear(i,o)




In [ ]:
# @title test_data_prep2
def test_data_prep2(X, sub_sample):

    total_X = None


    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:1000]
    print('Shape of X after trimming:',X.shape)

    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)


    total_X = X_max
    print('Shape of X after maxpooling:',total_X.shape)

    return total_X


def train_data_prep2(X,y,sub_sample,average,noise):

    total_X = None
    total_y = None

    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:1000]
    print('Shape of X after trimming:',X.shape)

    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)


    total_X = X_max
    total_y = y
    print('Shape of X after maxpooling:',total_X.shape)

    # Averaging + noise
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)

    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    print('Shape of X after averaging+noise and concatenating:',total_X.shape)

    # Subsampling

    for i in range(sub_sample):

        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)

        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))


    print('Shape of X after subsampling and concatenating:',total_X.shape)
    print('Shape of Y:',total_y.shape)
    return total_X,total_y

In [ ]:
# @title Data loading
data_root = "../project_data/project/"
if GOOGLE:
    data_root = "/content/drive/MyDrive/project/"
data_transform =  lambda x: x.reshape(1,x.shape[0],x.shape[1]) # fft transformation
data_transform = lambda x: abs(np.fft.fft(x))
data_transform = lambda x : scale_high_freq(x, 0.06, 0.4)
data_transform =  lambda x: x.reshape(1, x.shape[0],x.shape[1]).permute(1,2,0) # tonmoyfication2
data_transform =  lambda x: (x.reshape(1, x.shape[0],x.shape[1]))[:,0:18,:]
preprocess = lambda x,y:train_data_prep2(x,y,2,2,True)
# Create eeg dataset object
eeg_train = EEG_Data(data_root,
                              split='train',
                              preprocess=preprocess,
                              transform=data_transform)

eeg_val = EEG_Data(data_root,
                            split='val',
                            preprocess=preprocess,
                            transform=data_transform)
eeg_test = EEG_Data(data_root,
                            split='test',
                            preprocess=lambda x:test_data_prep2(x, 2),
                            transform=data_transform)
print("val split: ", len(eeg_val))
print("train split: ", len(eeg_train))
# Create the dataloaders
# Define the batch size and number of workers
batch_size = 64
num_workers=2
# Create DataLoader for training and validation sets
train_loader = DataLoader(eeg_train,
                          batch_size=batch_size,
                          num_workers=num_workers,
                        shuffle=True)
val_loader = DataLoader(eeg_val,
                        batch_size=batch_size,
                        num_workers=num_workers,
                        shuffle=False)
test_loader = DataLoader(eeg_test,
                        batch_size=batch_size,
                        num_workers=num_workers,
                        shuffle=False)

Shape of X after trimming: (1904, 22, 1000)
Shape of X after maxpooling: (1904, 22, 500)
Shape of X after averaging+noise and concatenating: (3808, 22, 500)
Shape of X after subsampling and concatenating: (7616, 22, 500)
Shape of Y: (7616,)
Shape of X after trimming: (211, 22, 1000)
Shape of X after maxpooling: (211, 22, 500)
Shape of X after averaging+noise and concatenating: (422, 22, 500)
Shape of X after subsampling and concatenating: (844, 22, 500)
Shape of Y: (844,)
Shape of X after trimming: (443, 22, 1000)
Shape of X after maxpooling: (443, 22, 500)
val split:  844
train split:  7616


In [ ]:
print(eeg_train[0][0].shape)
print(eeg_train[0][0][:,0:18,:].shape)

torch.Size([1, 22, 500])
torch.Size([1, 18, 500])


In [ ]:
# @title resnet
resnet =  nn.Sequential(
    ConvBlock(22,25,2,0),
    ConvBlock(25,50,5,0),
    ResBlock(50,128,5,0),
    nn.Flatten(start_dim=1),
    fc_block(5760, 500),
    nn.ReLU(),
    fc_block(500, 40),
    nn.ReLU(),
    fc_block(40,4)
)

In [ ]:
# @title resnetv2

resnetv2 =  nn.Sequential(
    nn.Conv1d(in_channels=22, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2),
    nn.BatchNorm1d(64),
    nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2),
    nn.BatchNorm1d(128),
    ResBlock(128,256,3,0),
    nn.MaxPool1d(kernel_size=2),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Flatten(start_dim=1),
    fc_block(12800, 512),
    nn.ReLU(),
    fc_block(512,4)
)


In [ ]:
resnetv21 =  nn.Sequential(
    nn.Conv1d(in_channels=22, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Dropout(0.8),
    nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Dropout(0.8),
    ResBlock(128,256,3,0.8),
    nn.AvgPool1d(3, stride=2),
    nn.Flatten(start_dim=1),
    nn.Linear(127744,40),
    nn.ReLU(),
    nn.BatchNorm1d(40),
    nn.Linear(40,4)
)

In [ ]:
# @title convnet3

convnet3 =  nn.Sequential(
    nn.Conv1d(in_channels=22, out_channels=20, kernel_size=2, padding=1, dilation=1),
    nn.ReLU(),
    nn.BatchNorm1d(20),
    nn.Dropout(0.6),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=2),
    nn.ReLU(),
    nn.BatchNorm1d(20),
    nn.Dropout(0.6),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=4),
    nn.ReLU(),
    nn.BatchNorm1d(20),
    nn.Dropout(0.6),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=8),
    nn.ReLU(),
    nn.BatchNorm1d(20),
    nn.Dropout(0.6),
    nn.Flatten(start_dim=1),
    fc_block(19860, 40),
    nn.BatchNorm1d(40),
    nn.ReLU(),
    fc_block(40,4)
)


In [ ]:
# @title convnet4
convnet4 =  nn.Sequential(
    nn.Conv1d(in_channels=22, out_channels=20, kernel_size=2, padding=1, dilation=1),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=2),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=4),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=8),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=16),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=32),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=64),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=128),
    nn.ReLU(),
    nn.Conv1d(in_channels=20, out_channels=20, kernel_size=2, padding=1, dilation=256),
    nn.ReLU(),
    nn.Flatten(start_dim=1),
    fc_block(10140, 40),
    nn.BatchNorm1d(40),
    nn.ReLU(),
    fc_block(40,4)
)

In [ ]:
# @title convnet5
convnet5 =  nn.Sequential(
    nn.Conv1d(in_channels=22, out_channels=22, kernel_size=2, padding=0), # 500
    nn.ReLU(),
    nn.Conv1d(in_channels=22, out_channels=22, kernel_size=4, padding=0), # 250
    nn.ReLU(),
    nn.Conv1d(in_channels=22, out_channels=22, kernel_size=8, padding=0), # 125
    nn.ReLU(),
    nn.Conv1d(in_channels=22, out_channels=22, kernel_size=1, padding=0), # ??
    nn.ReLU(),
    nn.Conv1d(in_channels=22, out_channels=22, kernel_size=1, padding=0), # ??
    nn.ReLU(),
    nn.Flatten(start_dim=1),
    fc_block(8558, 40),
    nn.BatchNorm1d(40),
    nn.ReLU(),
    fc_block(40,4)
)


In [ ]:
# @title LSTMClassifier

class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None

    def forward(self, x):
        x = x.permute(0,2,1)
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]


In [ ]:
# @title lstmnet
lstmnet =  nn.Sequential(
    nn.Conv1d(in_channels=22, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    ResBlock(128,256,3,0),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.AvgPool1d(3, stride=2),
    nn.Flatten(start_dim=1),
    nn.Linear(127744,40),
    nn.ReLU(),
    nn.Linear(40,4)
)

In [ ]:
# @title LSTMBlock
class LSTMBlock(nn.Module):
    def __init__(self, input_size, hidden_layer_dim, hidden_layer):
        super().__init__()
        self.LSTM = nn.LSTM(input_size, hidden_layer_dim, hidden_layer)

    def forward(self, x, **kwargs):
        # keey the residual connection here
        # x=x.reshape(x.shape[0],1, x.shape[1])
        out, (hn, cn) = self.LSTM(x)
        return out


In [ ]:
# @title convnet
convnet =  nn.Sequential(
    ConvBlock(22,25,5),
    ConvBlock(25,50,5),
    ConvBlock(50,100,5),
    ConvBlock(100,200,5),
    nn.Flatten(start_dim=1),
    fc_block(2600, 40),
    nn.BatchNorm1d(40),
    nn.ReLU(),
    LSTMBlock(40, 10, 2),
    fc_block(10,4)
)



In [ ]:
# @title HybridCNNLSTMModel
class HybridCNNLSTMModel(nn.Module):
    def __init__(self):
        super(HybridCNNLSTMModel, self).__init__()

        # Conv. block 1
        self.conv1 = nn.Conv2d(in_channels=22, out_channels=25, kernel_size=(5,5), padding='same')
        self.pool1 = nn.AvgPool2d(kernel_size=(3,1), stride=(3,1), padding=(1,0))
        self.norm1 = nn.BatchNorm2d(25)
        self.dropout1 = nn.Dropout(0.6)

        # Conv. block 2
        self.conv2 = nn.Conv2d(in_channels=25, out_channels=50, kernel_size=(5,5), padding='same')
        self.pool2 = nn.AvgPool2d(kernel_size=(3,1), stride=(3,1), padding=(1,0))
        self.norm2 = nn.BatchNorm2d(50)
        self.dropout2 = nn.Dropout(0.6)

        # Conv. block 3
        self.conv3 = nn.Conv2d(in_channels=50, out_channels=100, kernel_size=(5,5), padding='same')
        self.pool3 = nn.AvgPool2d(kernel_size=(3,1), stride=(3,1), padding=(1,0))
        self.norm3 = nn.BatchNorm2d(100)
        self.dropout3 = nn.Dropout(0.6)

        # Conv. block 4
        self.conv4 = nn.Conv2d(in_channels=100, out_channels=200, kernel_size=(5,5), padding='same')
        self.pool4 = nn.AvgPool2d(kernel_size=(3,1), stride=(3,1), padding=(1,0))
        self.norm4 = nn.BatchNorm2d(200)
        self.dropout4 = nn.Dropout(0.6)

        # FC+LSTM layers
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(200400, 40) # Adjust the input features according to your input size and architecture
        self.lstm = nn.LSTM(input_size=1, hidden_size=10, batch_first=True, dropout=0.4)

        # Output layer
        self.output = nn.Linear(10, 4) # Assuming the LSTM does not return sequences

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = self.pool1(x)
        x = self.norm1(x)
        x = self.dropout1(x)

        x = F.elu(self.conv2(x))
        x = self.pool2(x)
        x = self.norm2(x)
        x = self.dropout2(x)

        x = F.elu(self.conv3(x))
        x = self.pool3(x)
        x = self.norm3(x)
        x = self.dropout3(x)

        x = F.elu(self.conv4(x))
        x = self.pool4(x)
        x = self.norm4(x)
        x = self.dropout4(x)

        x = self.flatten(x)
        x = self.dense(x)
        x = x.view(-1, 40, 1) # Reshape for LSTM

        x, (hn, cn) = self.lstm(x)
        x = self.output(x[:, -1, :]) # Get the last sequence output for classification
        return x

In [ ]:
# This is EEGNet from https://arxiv.org/abs/1611.08024

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()

        self.F1 = 8
        self.F2 = 16
        self.D = 2

        # Conv2d(in,out,kernel,stride,padding,bias)
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, self.F1, (1, 64), padding=(0, 32), bias=False),
            nn.BatchNorm2d(self.F1)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(self.F1, self.D*self.F1, (18, 1), groups=self.F1, bias=False),
            nn.BatchNorm2d(self.D*self.F1),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(0.5)
        )

        self.Conv3 = nn.Sequential(
            nn.Conv2d(self.D*self.F1, self.D*self.F1, (1, 16), padding=(0, 8), groups=self.D*self.F1, bias=False),
            nn.Conv2d(self.D*self.F1, self.F2, (1, 1), bias=False),
            nn.BatchNorm2d(self.F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(0.5)
        )
        self.flatten = nn.Flatten(start_dim=1)
        self.classifier = nn.Linear(240, 4, bias=True)

    def forward(self, x):

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.Conv3(x)

        x = self.flatten(x)
        x = self.classifier(x)
        return x


In [ ]:
# @title TonmoyNN

# @title resnetv2

tonmoy =  nn.Sequential(
    nn.Conv2d(in_channels=22, out_channels=32, kernel_size=3, stride=1, padding=1),
    # nn.MaxPool2d(kernel_size=(3,1),padding=(1,0)),
    nn.ReLU(),
    nn.Dropout(0.6),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(kernel_size=(3,1),padding=(1,0)),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.Dropout(0.6),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=(3,1),padding=(1,0)),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(128),
    # nn.MaxPool2d(kernel_size=(3,1),padding=(1,0)),
    # nn.MaxPool2d(kernel_size=(3,1),padding=(1,0)),
    # nn.MaxPool2d(kernel_size=(3,2),padding=(1,0)),
    nn.MaxPool2d(kernel_size=(2,3),padding=(0,1)),
    nn.Flatten(start_dim=1),
    # LSTMBlock(80000, 22, 10),
    fc_block(7168, 22),
    nn.ReLU(),
    fc_block(22, 4),
)


In [ ]:
seed_everything(0)
# train_laoder.160transform =  lambda x: x.reshape(1,x.shape[0],x.shape[1]) # fft transformation
train_loader.transform = None # fft transformation


model = EEGNet()

model.to(device)

# Let's use the built-in optimizer for a full version of SGD optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# For loss function, your implementation and the built-in loss function should
# be almost identical.
criterion = nn.CrossEntropyLoss()

# Train the model
train(model,
      train_loader,
      test_loader,
      optimizer,
      criterion,
      device,
      num_epochs=60)

avg_loss, accuracy = evaluate(model, test_loader, criterion, device)

print("avg_loss", avg_loss)
print("accuracy", accuracy)



Epoch 1/60: 100%|██████████| 119/119 [00:03<00:00, 38.17it/s, loss=1.21]


Validation set: Average loss = 1.2793, Accuracy = 0.4289


Epoch 2/60: 100%|██████████| 119/119 [00:01<00:00, 62.38it/s, loss=1.11]


Validation set: Average loss = 1.1877, Accuracy = 0.4515


Epoch 3/60: 100%|██████████| 119/119 [00:01<00:00, 65.21it/s, loss=1.23]


...

Validation set: Average loss = 0.8098, Accuracy = 0.6840


Epoch 59/60: 100%|██████████| 119/119 [00:01<00:00, 69.49it/s, loss=0.774]


Validation set: Average loss = 0.8146, Accuracy = 0.6862


Epoch 60/60: 100%|██████████| 119/119 [00:01<00:00, 67.88it/s, loss=0.644]


Validation set: Average loss = 0.8217, Accuracy = 0.7111
avg_loss 0.8216522761753627
accuracy 0.7110609480812641


In [ ]:

avg_loss, accuracy = evaluate(model, test_loader, criterion, device)

print("avg_loss", avg_loss)
print("accuracy", accuracy)


avg_loss 0.8216522761753627
accuracy 0.7110609480812641
